In [2]:
import pandas as pd
import numpy as np
from tensorflow import keras
from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize as st

In [3]:
def tf_idf(text):
    sentences = st(text)
    vect = TfidfVectorizer(min_df=1)
    tfidf = vect.fit_transform(sentences)
    matrix = np.asarray((tfidf * tfidf.T).A)
    matrix.resize(50, 50)
    return matrix.flatten()

In [8]:
def flat_input(corpus, classes):
    docs = [title + '. ' + text for title, text in corpus]
    prep = [tf_idf(text) for text in docs]
    outp = [[1, 0] for i in classes if i == 'FAKE' else [0, 1]]
    return prep, outp

SyntaxError: invalid syntax (<ipython-input-8-2a1c9a7bf4ac>, line 4)

In [5]:
cols = ['ID', 'TITLE', 'TEXT', 'LABEL']
data = pd.read_csv("fake_or_real_news.csv", names=cols, header=0)

In [7]:
docs = [title + '. ' + text for title, text in zip(data[cols[1]], data[cols[2]])]

In [5]:
#news = manager.manage(data, cols[1:-1])
#classes = manager.classes(data, cols[-1])

Processed 0 6335
Processed 1 6335
Processed 2 6335
Processed 3 6335
Processed 4 6335
Processed 5 6335
Processed 6 6335
Processed 7 6335
Processed 8 6335
Processed 9 6335
Processed 10 6335
Processed 11 6335
Processed 12 6335
Processed 13 6335
Processed 14 6335
Processed 15 6335
Processed 16 6335
Processed 17 6335
Processed 18 6335
Processed 19 6335
Processed 20 6335
Processed 21 6335
Processed 22 6335
Processed 23 6335
Processed 24 6335
Processed 25 6335
Processed 26 6335
Processed 27 6335
Processed 28 6335
Processed 29 6335
Processed 30 6335
Processed 31 6335
Processed 32 6335
Processed 33 6335
Processed 34 6335
Processed 35 6335
Processed 36 6335
Processed 37 6335
Processed 38 6335
Processed 39 6335
Processed 40 6335
Processed 41 6335
Processed 42 6335
Processed 43 6335
Processed 44 6335
Processed 45 6335
Processed 46 6335
Processed 47 6335
Processed 48 6335
Processed 49 6335
Processed 50 6335
Processed 51 6335
Processed 52 6335
Processed 53 6335
Processed 54 6335
Processed 55 6335
Pr

Processed 437 6335
Processed 438 6335
Processed 439 6335
Processed 440 6335
Processed 441 6335
Processed 442 6335
Processed 443 6335
Processed 444 6335
Processed 445 6335
Processed 446 6335
Processed 447 6335
Processed 448 6335
Processed 449 6335
Processed 450 6335
Processed 451 6335
Processed 452 6335
Processed 453 6335
Processed 454 6335
Processed 455 6335
Processed 456 6335
Processed 457 6335
Processed 458 6335
Processed 459 6335
Processed 460 6335
Processed 461 6335
Processed 462 6335
Processed 463 6335
Processed 464 6335
Processed 465 6335
Processed 466 6335
Processed 467 6335
Processed 468 6335
Processed 469 6335
Processed 470 6335
Processed 471 6335
Processed 472 6335
Processed 473 6335
Processed 474 6335
Processed 475 6335
Processed 476 6335
Processed 477 6335
Processed 478 6335
Processed 479 6335
Processed 480 6335
Processed 481 6335
Processed 482 6335
Processed 483 6335
Processed 484 6335
Processed 485 6335
Processed 486 6335
Processed 487 6335
Processed 488 6335
Processed 48

Processed 871 6335
Processed 872 6335
Processed 873 6335
Processed 874 6335
Processed 875 6335
Processed 876 6335
Processed 877 6335
Processed 878 6335
Processed 879 6335
Processed 880 6335
Processed 881 6335
Processed 882 6335
Processed 883 6335
Processed 884 6335
Processed 885 6335
Processed 886 6335
Processed 887 6335
Processed 888 6335
Processed 889 6335
Processed 890 6335
Processed 891 6335
Processed 892 6335
Processed 893 6335
Processed 894 6335
Processed 895 6335
Processed 896 6335
Processed 897 6335
Processed 898 6335
Processed 899 6335
Processed 900 6335
Processed 901 6335
Processed 902 6335
Processed 903 6335
Processed 904 6335
Processed 905 6335
Processed 906 6335
Processed 907 6335
Processed 908 6335
Processed 909 6335
Processed 910 6335
Processed 911 6335
Processed 912 6335
Processed 913 6335
Processed 914 6335
Processed 915 6335
Processed 916 6335
Processed 917 6335
Processed 918 6335
Processed 919 6335
Processed 920 6335
Processed 921 6335
Processed 922 6335
Processed 92

Processed 1290 6335
Processed 1291 6335
Processed 1292 6335
Processed 1293 6335
Processed 1294 6335
Processed 1295 6335
Processed 1296 6335
Processed 1297 6335
Processed 1298 6335
Processed 1299 6335
Processed 1300 6335
Processed 1301 6335
Processed 1302 6335
Processed 1303 6335
Processed 1304 6335
Processed 1305 6335
Processed 1306 6335
Processed 1307 6335
Processed 1308 6335
Processed 1309 6335
Processed 1310 6335
Processed 1311 6335
Processed 1312 6335
Processed 1313 6335
Processed 1314 6335
Processed 1315 6335
Processed 1316 6335
Processed 1317 6335
Processed 1318 6335
Processed 1319 6335
Processed 1320 6335
Processed 1321 6335
Processed 1322 6335
Processed 1323 6335
Processed 1324 6335
Processed 1325 6335
Processed 1326 6335
Processed 1327 6335
Processed 1328 6335
Processed 1329 6335
Processed 1330 6335
Processed 1331 6335
Processed 1332 6335
Processed 1333 6335
Processed 1334 6335
Processed 1335 6335
Processed 1336 6335
Processed 1337 6335
Processed 1338 6335
Processed 1339 6335


KeyboardInterrupt: 

In [5]:
total = len(data[cols[-1]])
matrix = [news[i].to_list() for i in range(total // 2)]
#normal_matrix = manager.normal(matrix)
x_train = np.asarray([v for v in normal_matrix])
y_train = np.asarray([classes[i] for i in range(total // 2)])
#normal = manager.normal(np.asarray(matrix))


In [14]:
model = keras.Sequential()
model.add(keras.layers.Dense(units=300, activation='relu', input_dim=300))
model.add(keras.layers.Dense(units=100, activation='softmax'))
model.add(keras.layers.Dense(units=50, activation='softmax'))
model.add(keras.layers.Dense(units=10, activation='softmax'))
model.add(keras.layers.Dense(units=2, activation='softmax'))

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train, epochs=15, batch_size=32)

Epoch 1/15
3167/3167 [==============================] - 0s 119us/step - loss: 0.6934 - acc: 0.4948
Epoch 2/15
3167/3167 [==============================] - 0s 42us/step - loss: 0.6937 - acc: 0.5033
Epoch 3/15
3167/3167 [==============================] - 0s 41us/step - loss: 0.6939 - acc: 0.4913
Epoch 4/15
3167/3167 [==============================] - 0s 40us/step - loss: 0.6937 - acc: 0.4961
Epoch 5/15
3167/3167 [==============================] - 0s 41us/step - loss: 0.6935 - acc: 0.5008
Epoch 6/15
3167/3167 [==============================] - 0s 42us/step - loss: 0.6940 - acc: 0.4897
Epoch 7/15
3167/3167 [==============================] - 0s 42us/step - loss: 0.6935 - acc: 0.4935
Epoch 8/15
3167/3167 [==============================] - 0s 40us/step - loss: 0.6942 - acc: 0.4979
Epoch 9/15
3167/3167 [==============================] - 0s 41us/step - loss: 0.6937 - acc: 0.4847
Epoch 10/15
3167/3167 [==============================] - 0s 41us/step - loss: 0.6935 - acc: 0.4970
Epoch 11/15
3167/3

In [17]:
matrix_validation = [news[i].to_list() for i in range(total // 2, total)]
normal_matrix_validation = manager.normal(matrix_validation)
x_validate = np.asarray([v for v in normal_matrix_validation])
y_validate = np.asarray([classes[i] for i in range(total // 2, total)])

In [18]:
model.evaluate(x_validate, y_validate, batch_size=32)

3168/3168 [==============================] - 0s 31us/step


[0.6931817152283408, 0.4933712121212121]

In [19]:
result = model.predict(x_validate, batch_size=32)

In [20]:
for pred, real in zip(result, y_validate):
    print(str(pred) + ' ==> ' + str(real))

[0.5011947 0.4988053] ==> [0 1]
[0.50119466 0.49880537] ==> [0 1]
[0.5011948 0.4988052] ==> [0 1]
[0.50119466 0.49880534] ==> [0 1]
[0.50119466 0.49880537] ==> [0 1]
[0.5011947  0.49880528] ==> [1 0]
[0.50119466 0.49880534] ==> [1 0]
[0.50119466 0.4988053 ] ==> [1 0]
[0.50119466 0.4988054 ] ==> [1 0]
[0.50119483 0.4988052 ] ==> [0 1]
[0.50119483 0.49880522] ==> [0 1]
[0.5011948  0.49880528] ==> [1 0]
[0.50119483 0.49880522] ==> [1 0]
[0.5011946  0.49880546] ==> [1 0]
[0.50119466 0.4988054 ] ==> [0 1]
[0.50119483 0.49880522] ==> [1 0]
[0.50119454 0.49880543] ==> [0 1]
[0.50119466 0.49880534] ==> [0 1]
[0.50119483 0.49880517] ==> [0 1]
[0.50119466 0.4988053 ] ==> [1 0]
[0.50119466 0.4988053 ] ==> [0 1]
[0.5011947  0.49880525] ==> [1 0]
[0.5011948  0.49880522] ==> [1 0]
[0.50119454 0.49880552] ==> [0 1]
[0.5011947  0.49880528] ==> [0 1]
[0.50119483 0.4988052 ] ==> [1 0]
[0.5011945 0.4988055] ==> [1 0]
[0.5011947  0.49880534] ==> [0 1]
[0.50119466 0.49880534] ==> [0 1]
[0.5011947  0.498805

[0.5011946 0.4988054] ==> [0 1]
[0.50119483 0.49880517] ==> [0 1]
[0.50119454 0.49880543] ==> [0 1]
[0.50119466 0.49880537] ==> [0 1]
[0.50119466 0.49880534] ==> [1 0]
[0.50119466 0.4988053 ] ==> [1 0]
[0.50119454 0.49880543] ==> [1 0]
[0.5011947  0.49880534] ==> [0 1]
[0.50119483 0.49880517] ==> [1 0]
[0.50119466 0.4988053 ] ==> [1 0]
[0.5011946  0.49880546] ==> [0 1]
[0.5011947  0.49880528] ==> [0 1]
[0.5011948  0.49880525] ==> [0 1]
[0.5011949 0.4988051] ==> [1 0]
[0.5011949  0.49880514] ==> [1 0]
[0.5011948  0.49880522] ==> [1 0]
[0.5011948  0.49880525] ==> [0 1]
[0.5011948  0.49880525] ==> [0 1]
[0.5011947  0.49880534] ==> [0 1]
[0.5011946  0.49880546] ==> [1 0]
[0.5011949  0.49880514] ==> [0 1]
[0.50119466 0.49880534] ==> [0 1]
[0.50119466 0.49880534] ==> [1 0]
[0.5011947  0.49880528] ==> [0 1]
[0.5011947 0.4988053] ==> [1 0]
[0.50119466 0.49880534] ==> [1 0]
[0.50119466 0.49880537] ==> [1 0]
[0.50119454 0.49880546] ==> [1 0]
[0.5011947  0.49880534] ==> [0 1]
[0.5011948  0.498805